In [2]:
import random

In [25]:
def read_city_files(name_file, distance_file):
    """
    This function takes the file location for the city names file and
    the file location for the distances file and outputs two lists for
    using in other functions.

    Parameters:

    name_file (string): file containing the names of n cities
    distance_file (string): file containing an nxn matrix of distances between cities

    Returns:
    list of names and an nxn matrix of distances between cities
    """

    with open(name_file) as file:

        city_names = []
        for line in file:
            city_names.append(line.strip())

    with open(distance_file) as file:

        distances = []
        for line in file:
            line = line.strip()
            split_line = line.split()
            for i in range(len(split_line)):
                split_line[i] = float(split_line[i].strip())

            distances.append(split_line)

        return(city_names, distances)


# *Pseudocode For Evolutions*

    generate a list of the cities in a random order
    select a random city to move to that hasn't been visited
    if all cities have been visited select two cities and
    swap where they are in the visit order

    calculate distance (fitness metric)

    if distance is less than the current min save it as the current min

    if none of the offspring improve on the parent

    save parent as a local min

    mutate 5 times

    repeat mutations until new min is found

    once 3 local min are found return the most fit


In [4]:
def calc_distance(path_list, distances):
    """
    This function takes the path taken and distances between each city and
    calculates the total distance between the cities

    Parameters:
    path_list (list[int]): a list of cities visited in order
    distances (list[list[float]]): an nxn matrix where the distance of city i and city j
    is at (i,j) and (j,i)

    Returns:
    float: the total distance of the input path:
    """

    total_distance = 0
    # Sums distance of ith city visited to i+1th city visited except the last city to the original city

    for i in range(len(path_list) - 1):
        current_city = path_list[i]
        next_city = path_list[i + 1]
        total_distance += distances[current_city][next_city]

    # adds the distance of the last city to the original city
    total_distance += distances[path_list[-1]][path_list[0]]

    return total_distance



In [5]:
def tsp_mutate(path):
    """
    takes in a path, samples two of the cities and returns a path where those cities are swapped in the order
    (Used in tsp_evolution)
    Parameters:
    path_list (list[int]): a list of cities visited in order
    Returns:
    path_list (list[int]): a list of cities visited in order where two cities locations are swapped
    """

    cities_to_swap = random.sample(path, 2)

    city_1_idx = path.index(cities_to_swap[0])
    city_2_idx = path.index(cities_to_swap[1])

    path[city_1_idx] = cities_to_swap[1]
    path[city_2_idx] = cities_to_swap[0]

    return path

In [51]:
def tsp_evolution(name_file, distance_file):
    """
    takes in two file locations, feeds those through read_city_files to get the list of n city names
    and an nxn matrix of distances between them. Initiates a random path, then tries 100 random singular
    mutations on the path and saves the one with the shortest distance. From there it repeats that until
    it is unable to find a shorter path. It saves that path then mutates 5 consecutive on the fastest path.
    It finds 3 local minimums and returns the shortest distance and path.

    Parameters:
    name_file (string): file containing the names of n cities
    distance_file (string): file containing an nxn matrix of distances between cities
    Returns:
    min[0] (list[str]): a list of cities visited in order that is the shortest path found
    min[1] (float): distance of the shortest path
    """
    names, distances = read_city_files(name_file, distance_file)

    path = list(range(len(names)))

    random.shuffle(path)
    local_mins = []

    local_min_path = path
    local_min_dist = calc_distance(path, distances)


    while len(local_mins) < 3:
        # mutated is switched to true once a shorter path is found.
        # Thus if a mutation is not found, it will exit the while loop.
        mutated = True

        while mutated:
            mutated = False

            parent_path = local_min_path

            for i in range(1000):

                mutated_path = tsp_mutate(parent_path)
                mutated_dist = calc_distance(mutated_path, distances)

                if mutated_dist < local_min_dist:

                    mutated = True
                    local_min_dist = mutated_dist
                    local_min_path = mutated_path

        local_mins.append((local_min_path, local_min_dist))

        for i in range(100):
            local_min_path = tsp_mutate(local_min_path)

        local_min_dist = calc_distance(local_min_path, distances)

    min = local_mins[0]
    for min_path in local_mins:
        if min_path[1] < min[1]:
            min = min_path

    # Takes the list of integers and uses them to construct
    # the path with their respective names
    named_path = []
    for i in min[0]:
        named_path.append(names[i])
    min = (named_path, min[1])
    return min






In [15]:
tsp_evolution('Data/thirty_cities_names.txt', 'Data/thirty_cities_dist.txt')

['Azores', 'Baghdad', 'Berlin', 'Bombay', 'Buenos Aires', 'Cairo', 'Capetown', 'Chicago', 'Guam', 'Honolulu', 'Istanbul', 'Juneau', 'London', 'Manila', 'Melbourne', 'Mexico City', 'Montreal', 'Moscow', 'New Orleans', 'New York', 'Panama City', 'Paris', 'Rio de Janeiro', 'Rome', 'San Francisco', 'Santiago', 'Seattle', 'Shanghai', 'Sydney', 'Tokyo']


(['New Orleans',
  'Manila',
  'Tokyo',
  'Moscow',
  'Mexico City',
  'Rio de Janeiro',
  'Panama City',
  'Melbourne',
  'Baghdad',
  'Chicago',
  'Berlin',
  'Seattle',
  'Shanghai',
  'Rome',
  'Paris',
  'Montreal',
  'Buenos Aires',
  'Honolulu',
  'Capetown',
  'Juneau',
  'New York',
  'San Francisco',
  'Istanbul',
  'Bombay',
  'Azores',
  'Cairo',
  'Santiago',
  'Guam',
  'Sydney',
  'London'],
 1246.0)

# *Backtracking*
    tsp_backtrack takes in list of n cities, NxN array of distances
    base_city
    for city 1 from 1 to number citites

        for city 2 from city 1 to number of cities

        tsp_recursion takes in [city_1, base_city, city_2], list of all cities

    tsp_recursion takes in current cities in path and a list of all cities
        remove all cities currently in the path from the list of all cities
        if there are no cities to add to the path
            return the distance
        else:
            min_distance = -1
            min_path
            for all cities remaining

                append one at a time to the path and call tsp_recursion(appended_path, list of all cities)
                current_path, current_distance = tsp_recursion(appended_path, list of all cities)
                if min_distance = -1 or min_distance > current_distance
                    min_distance = current_distance
                    min_path = current_path

    return(min_path, min_distance)




['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon', 'Zeta', 'Eta']


In [18]:
def tsp_recursion(seed_path, remaining_cities, distances):
    """
    A recursive function for tsp_backtrack, it takes in a list of cities visited in order and
    a list of cities yet to be visited. If there are no more remaining cities, it calculates the
    distance of the path and returns that. Otherwise, for each remaining city it appends the remaining
    city to the seed_path and removes the city from the remaining cities and feeds the new_seed_path,
    new_remaining_cities and distances to a tsp_recursion (new_seed_path will only have one additional city
    at a time). The function saves the shortest distance and path and returns that.

    Parameters:
    seed_path (list[int]): list of cities currently visited in order
    remaining_cities (list[int]): list of cities not yet visited in order
    distances (list[list[float]]): an nxn matrix where the distance of city i and city j)

    Returns:
    min_path (list[int]): a list of cities visited in order that is the shortest path found
    min_distance (float): distance of the shortest path
    """

    if len(remaining_cities) == 0:
        distance = calc_distance(seed_path, distances)
        return seed_path, distance
    else:
        min_path = None
        min_distance = -1

        for i in remaining_cities:

            # for each city makes a list of remaining cities without
            # a singular city
            filtered_list = remaining_cities.copy()
            filtered_list.remove(i)

            # adds that city to a new seed path
            partial_path = seed_path.copy()
            partial_path.append(i)
            path, dist = tsp_recursion(partial_path, filtered_list, distances)

            # if the distance of the seed path is shorter
            # than each other possible new seed path
            # save it as the min_path and min_distance
            if (dist < min_distance) or (min_distance == -1):
                min_path = path
                min_distance = dist

        return min_path, min_distance

In [19]:
def tsp_backtrack(name_file, distance_file):
    """
    The seed function for tsp_recursive. It creates all potential seed "directions" as to allow for recursion
    to check each path without checking the same path more than once. It calls tsp_recursion for each seed, saving
    the one with the shortest distance and path and returns those.

    Parameters:
    name_file (string): file containing the names of n cities
    distance_file (string): file containing an nxn matrix of distances between cities

    Returns:
    named_path (list[int]): a list of city names visited in order that is the shortest path found
    min_distance (float): distance of the shortest path
    """
    names, distances = read_city_files(name_file, distance_file)

    list_of_cities = list(range(len(names)))

    base_city = list_of_cities[0]

    min_path, min_dist = None, None

    # creates each unique seed path because city_2 starts as the city directly after city_1
    for city_1 in range(1, len(list_of_cities)):
        for city_2 in range(city_1+1, len(list_of_cities)):
            remaining_cities_list = list_of_cities.copy()

            remaining_cities_list.remove(base_city)
            remaining_cities_list.remove(list_of_cities[city_1])
            remaining_cities_list.remove(list_of_cities[city_2])

            seed_path = [city_1, base_city, city_2]
            path, dist = tsp_recursion(seed_path, remaining_cities_list, distances)

            if min_dist == None or dist < min_dist:
                min_dist = dist
                min_path = path

    named_path = []
    for i in min_path:
        named_path.append(names[i])

    return named_path, min_dist

In [64]:
min_path, min_dist = tsp_evolution('Data/thirty_cities_names.txt'
                                   , 'Data/thirty_cities_dist.txt')

print(f"The shortest path found is {min_path}\n"
      f"The shortest distance found is {min_dist}")

The shortest path found is ['Juneau', 'Sydney', 'Rio de Janeiro', 'Seattle', 'Moscow', 'London', 'New York', 'Honolulu', 'Montreal', 'Tokyo', 'Istanbul', 'Bombay', 'Guam', 'Baghdad', 'Panama City', 'New Orleans', 'Rome', 'Manila', 'Azores', 'Paris', 'Shanghai', 'Buenos Aires', 'Chicago', 'Cairo', 'Mexico City', 'San Francisco', 'Melbourne', 'Capetown', 'Santiago', 'Berlin']
The shortest distance found is 1252.0


In [65]:
min_path, min_dist = tsp_backtrack('Data/seven_cities_names.txt', 'Data/seven_cities_dist.txt')
print(f"The shortest path found is {min_path}\n"
      f"The shortest distance found is {min_dist}")

The shortest path found is ['Epsilon', 'Alpha', 'Eta', 'Beta', 'Zeta', 'Delta', 'Gamma']
The shortest distance found is 106.4


In [26]:
min_path

['Epsilon', 'Alpha', 'Eta', 'Beta', 'Zeta', 'Delta', 'Gamma']

In [27]:
min_dist

106.4